# Heat Equation in 2D with scikit-fem - **Student Exercise**

**Goal**: Apply your knowledge from the 1D Poisson problem to solve the 2D heat equation.

We solve the steady-state heat equation (Laplace equation) on a unit square:
$$-\Delta u = 0,\quad (x,y)\in(0,1)^2$$

with Dirichlet boundary conditions:
- $u(x,0) = 1$ (bottom edge: hot)
- $u(x,1) = 0$ (top edge: cold)
- $u(0,y) = 0$ (left edge: cold)
- $u(1,y) = 0$ (right edge: cold)

**Your Task**: Complete the missing parts based on what you learned from the 1D example!

## Step 1: Import Required Libraries

We import the necessary libraries for 2D finite element computations:
- 2D mesh and triangular elements
- Plotting tools for 2D visualization

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import skfem as fem
from skfem.helpers import *

## Step 2: Define Mesh and Function Space

Here we create a 2D triangular mesh on the unit square [0,1]×[0,1]:
- `MeshTri.init_tensor` creates a structured triangular mesh
- `ElementTriP1` defines linear (P1) finite elements on triangles
- The mesh is refined to have reasonable resolution

In [ ]:
# Create a structured triangular mesh on unit square
n = 20  # number of subdivisions per direction
mesh = fem.MeshTri.init_tensor(np.linspace(0, 1, n+1), np.linspace(0, 1, n+1))
V = fem.Basis(mesh, fem.ElementTriP1())

print(f"Mesh has {mesh.p.shape[1]} nodes and {mesh.t.shape[1]} triangles")

# Visualize the mesh (coarse version for clarity)
if n <= 10:
    plt.figure(figsize=(6,6))
    mesh.plot()
    plt.title('Triangular Mesh')
    plt.axis('equal')
    plt.show()

## Step 3: Define Weak Form (Variational Formulation) - **YOUR TURN!**

**Background**: In 1D, you learned that $-u'' = f$ becomes the weak form:
$$\int_0^1 u' v' \, dx = \int_0^1 f \cdot v \, dx$$

**Questions to Guide You**:
1. **From 1D to 2D**: If $-u''$ becomes $-\Delta u = -(\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2})$ in 2D, what should the weak form look like?
2. **Gradient dot product**: In 1D we had $u'v'$. In 2D, we need $\nabla u \cdot \nabla v$. What does this mean?
   - $\nabla u = (\frac{\partial u}{\partial x}, \frac{\partial u}{\partial y})$
   - $\nabla v = (\frac{\partial v}{\partial x}, \frac{\partial v}{\partial y})$
   - $\nabla u \cdot \nabla v = ?$
3. **Source term**: Our equation is $-\Delta u = 0$. What should the right-hand side integral be?

**The weak form you need to implement**:
$$\int_{\Omega} \nabla u \cdot \nabla v \, dx dy = \int_{\Omega} 0 \cdot v \, dx dy \quad \forall v$$

In [ ]:
# TASK 1: Complete the bilinear form
# Hint: In 1D you used grad(u) * grad(v). In 2D, you need the dot product of gradients.
# The function dot(grad(u), grad(v)) computes ∇u · ∇v in any dimension
@fem.BilinearForm
def a(u, v, _):
    return _______  # FILL IN: What goes here for ∫ ∇u · ∇v dx dy?

# TASK 2: Complete the linear form
# Hint: The equation is -∆u = 0, so the right-hand side is f = 0
@fem.LinearForm  
def L(v, _):
    return _______  # FILL IN: What should this be for ∫ 0 * v dx dy?

## Step 4: Assembly

We assemble the stiffness matrix and load vector. Since we have no source term, the load vector will be zero (before applying boundary conditions).

In [ ]:
# Assemble stiffness matrix and load vector
A = a.assemble(V)
b = L.assemble(V)

print(f"System size: {A.shape[0]} x {A.shape[1]}")
print(f"Load vector norm: {np.linalg.norm(b):.2e} (should be ~0)")

## Step 5: Apply Boundary Conditions and Solve - **YOUR TURN!**

**Background**: In 1D, you applied boundary conditions by:
1. Finding boundary node indices (first and last nodes)
2. Setting matrix rows: `A[i, :] = 0; A[i, i] = 1`
3. Setting RHS values: `b[i] = boundary_value`

**Questions to Guide You**:
1. **2D Boundaries**: In 1D, boundaries were 2 points. In 2D, boundaries are edges. How do we find all nodes on the boundary?
2. **Different Values**: We want different values on different edges:
   - Bottom edge (y=0): $u = 1$ (hot)
   - Other edges: $u = 0$ (cold)
3. **Same Pattern**: The matrix modification pattern is the same as 1D, just with more boundary nodes.

**Your Task**: Complete the boundary condition application below.

In [ ]:
# Get node coordinates (this is provided)
x, y = mesh.p[0], mesh.p[1]

# TASK 3: Find boundary nodes
# Hint: Use np.where() to find nodes where coordinates equal boundary values
# Use a small tolerance (tol = 1e-12) for floating-point comparison
tol = 1e-12
bottom = _______  # FILL IN: Find nodes where y ≈ 0 (hint: np.where(np.abs(y) < tol)[0])
top = _______     # FILL IN: Find nodes where y ≈ 1
left = _______    # FILL IN: Find nodes where x ≈ 0  
right = _______   # FILL IN: Find nodes where x ≈ 1

print(f"Boundary nodes: bottom={len(bottom)}, top={len(top)}, left={len(left)}, right={len(right)}")

# TASK 4: Apply boundary conditions
# Hint: Use the same pattern as in 1D: A[i,:] = 0; A[i,i] = 1; b[i] = value
A_dense = A.toarray()

# Bottom edge: u = 1 (hot)
for i in bottom:
    A_dense[i, :] = _______  # FILL IN: What should this row become?
    A_dense[i, i] = _______  # FILL IN: What should the diagonal element be?
    b[i] = _______          # FILL IN: What temperature do we want on the bottom?

# Other edges: u = 0 (cold)
for boundary_nodes in [top, left, right]:
    for i in boundary_nodes:
        A_dense[i, :] = _______  # FILL IN: Same pattern as above
        A_dense[i, i] = _______  # FILL IN: Same pattern as above
        b[i] = _______          # FILL IN: What temperature for cold edges?

# Solve the system
u = np.linalg.solve(A_dense, b)
print(f"Solution computed with {len(u)} degrees of freedom")
print(f"Solution range: [{u.min():.3f}, {u.max():.3f}]")

## Step 6: Visualization

We visualize the temperature distribution using:
- Contour plot showing isotherms (lines of constant temperature)
- Color map showing temperature variation
- 3D surface plot for better understanding of the solution

The solution shows heat diffusion from the hot bottom edge, with temperature decreasing toward the cold boundaries.

In [ ]:
# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Contour plot
ax1 = axes[0]
levels = np.linspace(0, 1, 11)
cs = ax1.tricontourf(x, y, u, levels=levels, cmap='hot')
ax1.tricontour(x, y, u, levels=levels, colors='black', alpha=0.3, linewidths=0.5)
plt.colorbar(cs, ax=ax1, label='Temperature')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_title('Temperature Distribution')
ax1.set_aspect('equal')

# 3D surface plot
ax2 = fig.add_subplot(122, projection='3d')
surf = ax2.plot_trisurf(x, y, u, cmap='hot', alpha=0.8)
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('Temperature')
ax2.set_title('3D Temperature Surface')
plt.colorbar(surf, ax=ax2, shrink=0.5)

plt.tight_layout()
plt.show()

# Temperature profile along centerline (x=0.5)
plt.figure(figsize=(8, 4))
center_idx = np.where(np.abs(x - 0.5) < 0.02)[0]  # nodes near x=0.5
center_y = y[center_idx]
center_u = u[center_idx]
sort_idx = np.argsort(center_y)

plt.plot(center_y[sort_idx], center_u[sort_idx], 'o-', linewidth=2, markersize=6)
plt.xlabel('y (height)')
plt.ylabel('Temperature')
plt.title('Temperature Profile along Centerline (x=0.5)')
plt.grid(True, alpha=0.3)
plt.show()

## Analysis and Insights

The solution shows several important features:

1. **Boundary conditions**: Temperature is 1 at the bottom and 0 on other edges
2. **Heat diffusion**: Temperature decreases smoothly from bottom to top
3. **Corner effects**: Sharp gradients near corners where boundary conditions change
4. **Symmetry**: The solution is symmetric about the vertical centerline

This is a classic example of steady-state heat conduction governed by Laplace's equation.

## Solution Check

**After completing the tasks above, your code should:**
1. **Task 1**: `return dot(grad(u), grad(v))`
2. **Task 2**: `return 0.0 * v`
3. **Task 3**: 
   - `bottom = np.where(np.abs(y) < tol)[0]`
   - `top = np.where(np.abs(y - 1.0) < tol)[0]`
   - `left = np.where(np.abs(x) < tol)[0]`
   - `right = np.where(np.abs(x - 1.0) < tol)[0]`
4. **Task 4**: For each boundary node `i`:
   - `A_dense[i, :] = 0`
   - `A_dense[i, i] = 1` 
   - `b[i] = 1.0` (bottom) or `b[i] = 0.0` (other edges)

### Experiments (After Completing Tasks)
- Change the boundary conditions (e.g., make left/right edges hot)
- Add a heat source by modifying the linear form: `return 1.0 * v`
- Try different mesh resolutions and observe convergence
- Compare with the 1D solution: what are the key differences?